In [1]:
import sys
sys.path.append("../")

In [2]:
from model.inference import *
from model.parameter import *
from model.structure import *
import os
import pandas as pd
import numpy as np

In [3]:

df = pd.read_csv('../data/heart.csv')
df.head()
df = df.rename(columns={'target': 'heartdisease'})
df.head()

#sex: sex (1 = male; 0 = female)
#cp: chest pain type
#-- Value 1: typical angina
#-- Value 2: atypical angina
#-- Value 3: non-anginal pain
#-- Value 4: asymptomatic
#chol: serum cholestoral in mg/dl
#fbs: (fasting blood sugar > 120 mg/dl) (1 = true; 0 = false)
#restecg: resting electrocardiographic results
#-- Value 0: normal
#-- Value 1: having ST-T wave abnormality (T wave inversions and/or ST elevation or depression of > 0.05 mV)
#-- Value 2: showing probable or definite left ventricular hypertrophy by Estes' criteria
#thalach: maximum heart rate achieved
#thalrest: resting heart rate

#oldpeak = ST depression induced by exercise relative to rest
#slope: the slope of the peak exercise ST segment
#-- Value 1: upsloping
#-- Value 2: flat
#-- Value 3: downsloping

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,heartdisease
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1


In [4]:
X_cols =['age', 'sex', 'cp', 'trestbps', 'chol', 'fbs','restecg', 'thalach', 'exang']
y_col = ['heartdisease']

df_filter = df.loc[:, X_cols + y_col]
df_filter = df_filter.replace('?', np.nan)


In [5]:


edges = [('age', 'trestbps'), ('age', 'fbs'), ('sex', 'trestbps'), ('sex', 'trestbps'), 
                       ('exang', 'trestbps'),('trestbps','heartdisease'),('fbs','heartdisease'),
                      ('heartdisease','restecg'),('heartdisease','thalach'),('heartdisease','chol')]


model = true_structure(edges)

### Parameter Estimation using Bayesian Estimator

In [6]:
#Learing CPDs using Bayesian Estimators

model, cpd = learn_model_parameters(model, df_filter, "BE",prior_type="BDeu", eq_sample_size=1000, get_cpd="age")
print(cpd)

+---------+-----------+
| age(29) | 0.019486  |
+---------+-----------+
| age(34) | 0.0202534 |
+---------+-----------+
| age(35) | 0.0217884 |
+---------+-----------+
| age(37) | 0.0202534 |
+---------+-----------+
| age(38) | 0.0210209 |
+---------+-----------+
| age(39) | 0.0217884 |
+---------+-----------+
| age(40) | 0.0210209 |
+---------+-----------+
| age(41) | 0.0263931 |
+---------+-----------+
| age(42) | 0.0248582 |
+---------+-----------+
| age(43) | 0.0248582 |
+---------+-----------+
| age(44) | 0.0271606 |
+---------+-----------+
| age(45) | 0.0248582 |
+---------+-----------+
| age(46) | 0.0240907 |
+---------+-----------+
| age(47) | 0.0225558 |
+---------+-----------+
| age(48) | 0.0240907 |
+---------+-----------+
| age(49) | 0.0225558 |
+---------+-----------+
| age(50) | 0.0240907 |
+---------+-----------+
| age(51) | 0.027928  |
+---------+-----------+
| age(52) | 0.0286955 |
+---------+-----------+
| age(53) | 0.0248582 |
+---------+-----------+
| age(54) | 0.03

### Inference - Computing probability of variable

What is probability of heart disease given someone is male ?


In [7]:
# Computing the probability of bronc given smoke.
q = infer_variable_cpd(model, ['heartdisease'], {'sex': 1})
print(q)

Finding Elimination Order: : 100%|███████████████| 4/4 [00:00<00:00, 493.46it/s]

Eliminating: exang: 100%|████████████████████████| 4/4 [00:00<00:00, 157.17it/s]

+-----------------+---------------------+
| heartdisease    |   phi(heartdisease) |
+=================+=====================+
| heartdisease(0) |              0.4903 |
+-----------------+---------------------+
| heartdisease(1) |              0.5097 |
+-----------------+---------------------+
